In [2]:
import os
import json
import gdown

In [6]:
#url = 'https://drive.google.com/uc?id=1XKQJCptrk1NtOgvg-A2DO4XySKwnxkRV'
#out_path = 'annotated_datasets_for_yolo.zip' 
#gdown.download(url, out_path, quiet=False)

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1XKQJCptrk1NtOgvg-A2DO4XySKwnxkRV' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XKQJCptrk1NtOgvg-A2DO4XySKwnxkRV" -O annotated_datasets_for_yolo.zip && rm -rf /tmp/cookies.txt

--2023-05-01 12:46:53--  https://docs.google.com/uc?export=download&confirm=t&id=1XKQJCptrk1NtOgvg-A2DO4XySKwnxkRV
Resolving docs.google.com (docs.google.com)... 142.251.36.46, 2a00:1450:400e:802::200e
Connecting to docs.google.com (docs.google.com)|142.251.36.46|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-04-4s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sotdhua12kabrvjdp2ht11ggsooqhn68/1682945175000/03830746775410815741/*/1XKQJCptrk1NtOgvg-A2DO4XySKwnxkRV?e=download&uuid=6ad4a9e4-a775-41fd-a7f8-0719bbf8586d [following]
--2023-05-01 12:46:53--  https://doc-04-4s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/sotdhua12kabrvjdp2ht11ggsooqhn68/1682945175000/03830746775410815741/*/1XKQJCptrk1NtOgvg-A2DO4XySKwnxkRV?e=download&uuid=6ad4a9e4-a775-41fd-a7f8-0719bbf8586d
Resolving doc-04-4s-docs.googleusercontent.com (doc-04-4s-docs.googleusercontent.com)... 142.250.179.129, 2a00:1450:40

In [7]:
root_dir = os.path.join(*["/", "paperspace", "GitHub", "yolov7"])
!cd {root_dir}
!unzip annotated_datasets_for_yolo.zip -d annotated_datasets_for_yolo

/bin/bash: line 0: cd: /paperspace/GitHub/yolov7: No such file or directory
Archive:  annotated_datasets_for_yolo.zip
   creating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/
   creating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_white/
   creating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/
   creating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/images/
  inflating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/images/Garg_DSC_0362.JPG  
  inflating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/images/Gler_glerafd(50).JPG  
  inflating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/images/Gler_tesista_65a90f2f-DSC_0530-min.JPG  
  inflating: annotated_datasets_for_yolo/annotated_datasets_for_yolo/Esca_FD_multivariety_w

FUNCTION DEFINITIONS

In [8]:
def initialize_source_data(source_data):
    for key in source_data.keys():
        print(f"------------ {source_data[key]['description']} ------------")
        fldr = source_data[key]['images']['folder_name']
        print(f"Folder: {fldr}")
        source_data[key]['images']['content'] = os.listdir(fldr)
        source_data[key]['images']['content'].sort()
        source_data[key]['images']['n_elts'] = len(source_data[key]['images']['content'])
        if source_data[key]['images']['n_elts'] > 0:
            print(f"Content: [{source_data[key]['images']['content'][0]}, ...]")
            print(f"N. elements: {source_data[key]['images']['n_elts']}")
        else:
            print(f"Content: empty.")
        
        fldr = source_data[key]['labels']['folder_name']
        print(f"Folder: {fldr}")
        source_data[key]['labels']['content'] = os.listdir(fldr)
        source_data[key]['labels']['content'].sort()
        source_data[key]['labels']['n_elts'] = len(source_data[key]['labels']['content'])
        if source_data[key]['labels']['n_elts'] > 0:
            print(f"Content: [{source_data[key]['labels']['content'][0]}, ...]")
            print(f"N. elements: {source_data[key]['labels']['n_elts']}")
        else:
            print(f"Content: empty.")


def data_sanity_check(source_data):
    for key in source_data:
        lbls = source_data[key]['labels']['content']
        imgs = source_data[key]['images']['content']
        n = len(lbls)
        if n != len(imgs):
            print(f"Comparing {source_data[key]['labels']['folder_name']} and {source_data[key]['images']['folder_name']}:")
            print(f"Different number of elements.")
            return False
        for i in range(n):# we run trough the numberings of the labels
            if lbls[i][:-3] != imgs[i][:-3]:
                print(f"Comparing {source_data[key]['labels']['folder_name']} and {source_data[key]['images']['folder_name']}:")
                print(f"Some file has different name.")
                return False
    return True

def find_missing_images(source_data):# to be used when there are more labels than images
    result = []
    for key in source_data:
        lbls = source_data[key]['labels']['content']
        imgs = source_data[key]['images']['content']
        for el in lbls:
            if not f"{el[:-4]}.jpg" in imgs and not f"{el[:-4]}.JPG" in imgs:
                result.append(f"{el[:-4]}.jpg")
    return result

In [11]:
#works correctly if the current woring directory is "/paperspace/GitHub/yolov7"
base_dir = os.path.join(*['annotated_datasets_for_yolo'])
Esca_FD_multivariety_white = {
    'test': {
            'description': 'Test set',
            'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'test']),
            'images': 
            {
                'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'test', 'images']),
                'content': [],
                'n_elts': 0
            },
            'labels': 
            {
                'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'test', 'labels']),
                'content': [],
                'n_elts': 0
            }
    },
    'train': {
            'description': 'Training set',
            'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'train']),
            'images': 
            {
                'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'train', 'images']),
                'content': [],
                'n_elts': 0
            },
            'labels': 
            {
                'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'train', 'labels']),
                'content': [],
                'n_elts': 0
            }
    },
    'val': {
            'description': 'Validation set',
            'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'val']),
            'images': 
            {
                'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'val', 'images']),
                'content': [],
                'n_elts': 0
            },
            'labels': 
            {
                'folder_name': os.path.join(*[base_dir, 'Esca_FD_multivariety_white', 'val', 'labels']),
                'content': [],
                'n_elts': 0
            }
    }
}


initialize_source_data(Esca_FD_multivariety_white)
test = data_sanity_check(Esca_FD_multivariety_white)
test_message = ""
if test:
    test_message = "ok."
else:
    test_message = "not ok."

print(f"Checking dataset Esca_FD_multivariety_white: {test_message}")

------------ Test set ------------
Folder: annotated_datasets_for_yolo/Esca_FD_multivariety_white/test/images
Content: [Char_DSC_00333.JPG, ...]
N. elements: 69
Folder: annotated_datasets_for_yolo/Esca_FD_multivariety_white/test/labels
Content: [Char_DSC_00333.txt, ...]
N. elements: 69
------------ Training set ------------
Folder: annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/images
Content: [Char_DSC_0004.JPG, ...]
N. elements: 548
Folder: annotated_datasets_for_yolo/Esca_FD_multivariety_white/train/labels
Content: [Char_DSC_0004.txt, ...]
N. elements: 548
------------ Validation set ------------
Folder: annotated_datasets_for_yolo/Esca_FD_multivariety_white/val/images
Content: [Char_DSC_0015.JPG, ...]
N. elements: 68
Folder: annotated_datasets_for_yolo/Esca_FD_multivariety_white/val/labels
Content: [Char_DSC_0015.txt, ...]
N. elements: 68
Checking dataset Esca_FD_multivariety_white: ok.


In [18]:
base_dir = os.path.join(*['annotated_datasets_for_yolo'])
FD_JBoulent = {
    'test': {
            'description': 'Test set',
            'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'test']),
            'images': 
            {
                'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'test', 'images']),
                'content': [],
                'n_elts': 0
            },
            'labels':
            {
                'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'test', 'labels']),
                'content': [],
                'n_elts': 0
            }
    },
    'train': {
            'description': 'Training set',
            'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'train']),
            'images': 
            {
                'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'train', 'images']),
                'content': [],
                'n_elts': 0
            },
            'labels': 
            {
                'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'train', 'labels']),
                'content': [],
                'n_elts': 0
            }
    },
    'val': {
        'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'val']),
            'description': 'Validation set',
            'images': 
            {
                'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'val', 'images']),
                'content': [],
                'n_elts': 0
            },
            'labels': 
            {
                'folder_name': os.path.join(*[base_dir, 'FD_JBoulent', 'val', 'labels']),
                'content': [],
                'n_elts': 0
            }
    }
}

initialize_source_data(FD_JBoulent)
test = data_sanity_check(FD_JBoulent)
test_message = ""
if test:
    test_message = "ok."
else:
    test_message = "not ok."

print(f"Checking dataset FD_JBoulent: {test_message}")

------------ Test set ------------
Folder: annotated_datasets_for_yolo/FD_JBoulent/test/images
Content: [TRAIN_004.jpg, ...]
N. elements: 90
Folder: annotated_datasets_for_yolo/FD_JBoulent/test/labels
Content: [TRAIN_004.txt, ...]
N. elements: 90
------------ Training set ------------
Folder: annotated_datasets_for_yolo/FD_JBoulent/train/images
Content: [TRAIN_001.jpg, ...]
N. elements: 718
Folder: annotated_datasets_for_yolo/FD_JBoulent/train/labels
Content: [TRAIN_001.txt, ...]
N. elements: 718
------------ Validation set ------------
Folder: annotated_datasets_for_yolo/FD_JBoulent/val/images
Content: [TRAIN_007.jpg, ...]
N. elements: 90
Folder: annotated_datasets_for_yolo/FD_JBoulent/val/labels
Content: [TRAIN_007.txt, ...]
N. elements: 90
Checking dataset FD_JBoulent: ok.


In [13]:
# All'inizio tutte le immagini erano in "/notebooks/annotated_datasets/FD_JBoulent/train/images"
# e tutte le etichette in "/notebooks/annotated_datasets/FD_JBoulent/train/labels"
# Così utilizziamo la funzione "train_test_split()" e "shutil.move()" per ripartire immagini ed etichette nelle tre sezioni
# train / test / val

import shutil
from sklearn.model_selection import train_test_split

train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.1

dataX_folder = "/home/paperspace/GitHub/yolov7/annotated_datasets_for_yolo/FD_JBoulent/images"
dataX = os.listdir(dataX_folder)
dataX.sort()
dataY_folder = "/home/paperspace/GitHub/yolov7/annotated_datasets_for_yolo/FD_JBoulent/labels"
dataY = os.listdir(dataY_folder)
dataY.sort()

x_train, x_test, y_train, y_test = train_test_split(dataX, dataY, test_size=1 - train_ratio)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio))

In [14]:
for pair in zip(x_val, y_val):
    print(f"{pair[0]}  ;  {pair[1]}")

TRAIN_025.jpg  ;  TRAIN_025.txt
TRAIN_868.jpg  ;  TRAIN_868.txt
TRAIN_890.jpg  ;  TRAIN_890.txt
TRAIN_050.jpg  ;  TRAIN_050.txt
TRAIN_436.jpg  ;  TRAIN_436.txt
TRAIN_884.jpg  ;  TRAIN_884.txt
TRAIN_857.jpg  ;  TRAIN_857.txt
TRAIN_091.jpg  ;  TRAIN_091.txt
TRAIN_094.jpg  ;  TRAIN_094.txt
TRAIN_258.jpg  ;  TRAIN_258.txt
TRAIN_246.jpg  ;  TRAIN_246.txt
TRAIN_474.jpg  ;  TRAIN_474.txt
TRAIN_486.jpg  ;  TRAIN_486.txt
TRAIN_618.jpg  ;  TRAIN_618.txt
TRAIN_607.jpg  ;  TRAIN_607.txt
TRAIN_548.jpg  ;  TRAIN_548.txt
TRAIN_192.jpg  ;  TRAIN_192.txt
TRAIN_035.jpg  ;  TRAIN_035.txt
TRAIN_547.jpg  ;  TRAIN_547.txt
TRAIN_164.jpg  ;  TRAIN_164.txt
TRAIN_372.jpg  ;  TRAIN_372.txt
TRAIN_415.jpg  ;  TRAIN_415.txt
TRAIN_608.jpg  ;  TRAIN_608.txt
TRAIN_813.jpg  ;  TRAIN_813.txt
TRAIN_321.jpg  ;  TRAIN_321.txt
TRAIN_680.jpg  ;  TRAIN_680.txt
TRAIN_838.jpg  ;  TRAIN_838.txt
TRAIN_836.jpg  ;  TRAIN_836.txt
TRAIN_538.jpg  ;  TRAIN_538.txt
TRAIN_466.jpg  ;  TRAIN_466.txt
TRAIN_126.jpg  ;  TRAIN_126.txt
TRAIN_24

In [15]:
#ACTUAL MOVEMENT OF FILES
for filename in x_test:
    old_path = os.path.join(*[dataX_folder, filename])
    new_path = os.path.join(*[FD_JBoulent['test']['images']['folder_name'], filename])
    shutil.move(old_path, new_path)

for filename in y_test:
    old_path = os.path.join(*[dataY_folder, filename])
    new_path = os.path.join(*[FD_JBoulent['test']['labels']['folder_name'], filename])
    shutil.move(old_path, new_path)

In [16]:
for filename in x_val:
    old_path = os.path.join(*[dataX_folder, filename])
    new_path = os.path.join(*[FD_JBoulent['val']['images']['folder_name'], filename])
    shutil.move(old_path, new_path)

for filename in y_val:
    old_path = os.path.join(*[dataY_folder, filename])
    new_path = os.path.join(*[FD_JBoulent['val']['labels']['folder_name'], filename])
    shutil.move(old_path, new_path)

In [17]:
for filename in x_train:
    old_path = os.path.join(*[dataX_folder, filename])
    new_path = os.path.join(*[FD_JBoulent['train']['images']['folder_name'], filename])
    shutil.move(old_path, new_path)

for filename in y_train:
    old_path = os.path.join(*[dataY_folder, filename])
    new_path = os.path.join(*[FD_JBoulent['train']['labels']['folder_name'], filename])
    shutil.move(old_path, new_path)